In [ ]:
from bs4 import BeautifulSoup
import subprocess
import pandas as pd
from funcs import tratamento_texto

In [ ]:
def extrair_eventos(ano_inicio, ano_fim):
  data = []
  for ano in range(ano_inicio, ano_fim+1):
    if ano < 1492:
      url = "https://www.onthisday.com/date"
    else:
      url = "https://www.onthisday.com/events/date"
    for pagina in range(1, 6):
      if pagina >= 2:
        curl_command = ["curl", f"{url}/{ano}?p={pagina}"]
      else:
        curl_command = ["curl", f"{url}/{ano}"]
      result = subprocess.run(curl_command, capture_output=True, text=True, encoding='utf-8')
      soup = BeautifulSoup(result.stdout, 'html.parser')

      try:
        year = soup.find('span', class_='year').get_text()
        highlights = soup.find_all('div', class_='section--highlight')
        for h in highlights:
          date = h.find('p').a.get_text()  
          event = h.get_text().replace(date, '', 1).strip()
          data.append({"Year": year, "Date": date, "Event": event})

        article = soup.find('article')
        event_list = article.find_all('li', class_=['event', 'person'])
        for e in event_list:
          date = None
          if e.b:
            date = e.b.get_text()  
          for cls in ['date', 'deathDate', 'birthDate']:
            tag = e.find('a', class_=cls)
            if tag:
              date = tag.get_text()
          event = e.get_text().replace(date, '', 1).strip()
          data.append({"Year": year, "Date": date, "Event": event})
      except:
        break
  return data

In [ ]:
df = pd.DataFrame(extrair_eventos(1, 2024))

In [ ]:
df

In [ ]:
coluna_texto = "Event"

In [ ]:
df["texto_tratado"] = df[coluna_texto].apply(tratamento_texto)

In [ ]:
df

In [ ]:
df.to_csv("dataset/events.csv", sep=";", index=False)